In [2]:
import pandas as pd
import requests
import os
import json

/Users/mdacko/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
DATASETS = {
    "allowance_events": "https://gist.githubusercontent.com/DaniModak/d0cdc441bc2cab2abdc5b37e45ca5cb4/raw/13ded757082f09740a0ca351f926b74c336206ab/allowance_events",
    "allowance_backend_table": "https://gist.githubusercontent.com/DaniModak/d0cdc441bc2cab2abdc5b37e45ca5cb4/raw/13ded757082f09740a0ca351f926b74c336206ab/allowance_backend_table",
    "payment_schedule_backend_table": "https://gist.githubusercontent.com/DaniModak/d0cdc441bc2cab2abdc5b37e45ca5cb4/raw/13ded757082f09740a0ca351f926b74c336206ab/payment_schedule_backend_table",
}

In [4]:
def download_file(url, dest_path):
    if not os.path.exists(dest_path):
        print(f"Downloading {url} to {dest_path}...")
        response = requests.get(url)
        response.raise_for_status()
        with open(dest_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {dest_path}.")
    else:
        print(f"{dest_path} already exists. Skipping download!")

def load_allowance_events(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return pd.json_normalize(data)

def load_allowance_backend(file_path):
    return pd.read_csv(file_path)

def load_payment_schedule(file_path):
    return pd.read_csv(file_path)

def prepare_datasets(data_dir="../data/input"):
    """Download datasets and load them as dataframes."""
    os.makedirs(data_dir, exist_ok=True) 

    for name, url in DATASETS.items():
        dest_path = os.path.join(data_dir, f"{name}.{'json' if 'allowance_events' in url else 'csv'}")
        download_file(url, dest_path) 

    allowance_events = load_allowance_events(os.path.join(data_dir, "allowance_events.json"))
    allowance_backend = load_allowance_backend(os.path.join(data_dir, "allowance_backend_table.csv"))
    payment_schedule = load_payment_schedule(os.path.join(data_dir, "payment_schedule_backend_table.csv"))

    return allowance_events, allowance_backend, payment_schedule

In [5]:
events, backend, schedule = prepare_datasets()
print("Allowance Events:")
print(events.head())
print("\nAllowance Backend Table:")
print(backend.head())
print("\nPayment Schedule Backend Table:")
print(schedule.head())

Downloaded ../data/allowance_events.json.
Downloaded ../data/allowance_backend_table.csv.
Downloaded ../data/payment_schedule_backend_table.csv.
Allowance Events:
                                user.id      event.timestamp  \
0  03c5ffe8-6cd2-434b-a1b2-c4006860d94c   2024-09-21 1:39:34   
1  03c5ffe8-6cd2-434b-a1b2-c4006860d94c  2024-09-22 19:38:22   
2  03c5ffe8-6cd2-434b-a1b2-c4006860d94c  2024-09-22 19:38:22   
3  05384674-4af8-4f70-8d8f-90e19b0db971   2024-10-06 6:32:28   
4  05384674-4af8-4f70-8d8f-90e19b0db971   2024-10-06 6:32:37   

          event.name allowance.scheduled.frequency allowance.scheduled.day  \
0  allowance.created                        weekly                  sunday   
1   allowance.edited                        weekly                  sunday   
2   allowance.edited                        weekly                  sunday   
3  allowance.created                      biweekly                  friday   
4   allowance.edited                        weekly            